In [38]:
import pandas as pd
import pandas_profiling
import numpy as np
import json
from jinja2 import Environment, FileSystemLoader

In [39]:
%%javascript
require.config({
paths: {
d3: "https://d3js.org/d3.v5.min",
}
});

require(["d3"], function(d3) {
window.d3 = d3;
});

<IPython.core.display.Javascript object>

In [40]:
def build_data(df):
    count = []
    considered_cols = []
    last_index = 0
    keys = {}

    df.replace(to_replace=["na", "?", np.nan, "missing", "not available",
                           "n/a", "missing value"], value=np.nan, inplace=True)

    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='ignore')

    for col in df.columns:
        if df[col].dtype == "object":
            df[col].replace(to_replace=["na", "?", np.nan, "missing", "not available",
                                        "n/a", "missing value"], value="missing value", inplace=True)
            if df[col].unique().size < 25:
                considered_cols.append(col)
                keys[col] = {"values": df[col].unique().tolist(
                ), "start_i": last_index, "end_i": last_index + df[col].unique().size}
                last_index = last_index + df[col].unique().size
                for item in df[col].unique():
                    count.append([col, item])
    data_array = np.zeros((len(count), len(count)))

    for row in range(df.shape[0]):
        for col in range(df.shape[1]):
            for i in range(col+1, df.shape[1]):
                if df.columns[col] in considered_cols and df.columns[i] in considered_cols:
                    data_array[count.index([df.columns[col], df.iloc[row, col]]), count.index(
                        [df.columns[i], df.iloc[row, i]])] += 1

    data_array += np.transpose(data_array)
    return keys, data_array.tolist()

In [41]:
def table_data_format(df_orig):
    metadata = []
    df = df_orig
    df.replace(to_replace=["na", "?", np.nan, "missing", "not available",
                           "n/a", "missing value"], value=np.nan, inplace=True)

    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='ignore')

    for idx, col in enumerate(df.columns):
        helper = {}
        key = col
        if len(key) > 15:
            key = col[:15]

        helper['ascend'] = 0
        helper['name'] = key
        if(df.dtypes[col] == "object"):
            df[col].replace(to_replace=["na", "?", np.nan, "missing", "not available",
                                        "n/a", "missing value"], value="missing value", inplace=True)
            helper['datatype'] = 'string'
            helper['values'] = df[col].unique().tolist()
        elif(df.dtypes[col] == "int64"):
            helper['datatype'] = 'int'
            helper['min_val'] = df[col].min()
            helper['max_val'] = df[col].max()
        elif(df.dtypes[col] == "float64"):
            helper['datatype'] = 'float'
            helper['min_val'] = df[col].min()
            helper['max_val'] = df[col].max()
        metadata.append(helper)

    df.replace(to_replace=["na", "?", np.nan, "missing", "not available",
                           "n/a", "missing value"], value="missing value", inplace=True)
    data_array = []
    for row in range(df.shape[0]):
        aux = []
        for col in range(df.shape[1]):
            aux.append(df.iloc[row, col])
        data_array.append(aux)
    return metadata, data_array

In [42]:
columns = ["symboling", "normalized-losses", "make", "fuel-type", "aspiration", "num-of-doors", "body-style", "drive-wheels", "engine-location", "wheel-base", "length", "width", "height",
           "curb-weight", "engine-type", "num-of-cylinders", "engine-size", "fuel-system", "bore", "stroke", "compression-ratio", "horsepower", "peak-rpm", "city-mpg", "highway-mpg", "price"]

df = pd.read_csv('imports-85.data', names=columns)

for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='ignore')

df.replace(to_replace=["na", "?", np.nan, "missing", "not available","n/a", "missing value"], value=np.nan, inplace=True)

In [ ]:
pandas_profiling.ProfileReport(df)

In [ ]:
pandas_profiling.ProfileReport(df).to_file("../../../../Desktop/example.html")

In [ ]:
table_data_format(df)

In [46]:
fields = [
          'TP_SEXO', 'TP_COR_RACA', 'TP_ESCOLA', 'TP_ENSINO', 
          'IN_TREINEIRO', 'TP_PRESENCA_CN', 
          "TP_PRESENCA_CH", "TP_PRESENCA_LC", 'TP_PRESENCA_MT', "NU_NOTA_CN", 'NU_NOTA_CH', 
          'NU_NOTA_LC', 'NU_NOTA_MT', 'TP_LINGUA', 'TP_STATUS_REDACAO', 'NU_NOTA_REDACAO', 'Q006', 'Q026',
          'NU_IDADE',
          'CO_MUNICIPIO_RESIDENCIA'
         ]
df = pd.read_csv('C:\\Users\\Lucas\\Downloads\\microdados_enem2018\\DADOS\\MICRODADOS_ENEM_2018.csv', 
                 sep=';', encoding='latin-1', skipinitialspace=True, usecols=fields)


def region_of_residency(a):
    if a == 4314902:
        return 'Porto Alegre'
    return 'drop'
        
df['CO_MUNICIPIO_RESIDENCIA'] = list(map(region_of_residency,df['CO_MUNICIPIO_RESIDENCIA']))

df.drop(df[
    (df['IN_TREINEIRO'] == 1) |
    (df['TP_PRESENCA_CN'] != 1) |
    (df['TP_PRESENCA_CH'] != 1) |
    (df['TP_PRESENCA_LC'] != 1) |
    (df['TP_PRESENCA_MT'] != 1) |
    (df['TP_STATUS_REDACAO'] != 1) |
    (df['Q026'] != 'B') |
    (df['CO_MUNICIPIO_RESIDENCIA'] == 'drop') |
    (df['TP_ENSINO'] != 1) |
    (df['TP_ESCOLA'] == 3)
    ].index, inplace=True)



df['NOTA_FINAL'] = list(map(lambda a,b,c,d, e: (a + b + c + d + e)/5, df['NU_NOTA_CN'], 
                            df['NU_NOTA_CH'], df['NU_NOTA_LC'], df['NU_NOTA_MT'], df['NU_NOTA_REDACAO']))
df["TP_COR_RACA"].replace(to_replace ={0:np.nan,1:"Branca",2:'Preta',3:"Parda", 4:'Amarela', 5:'Indigena'},
                          inplace=True)
df.drop(columns=['IN_TREINEIRO', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'TP_PRESENCA_CN', 
                 'TP_STATUS_REDACAO', 'Q026', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 
                 'NU_NOTA_REDACAO', 'TP_ENSINO'], inplace=True)
df["TP_ESCOLA"].replace(to_replace ={1:np.nan, 2:'Publica', 3:"Exterior", 4:'Privada'}, inplace=True)
df["TP_LINGUA"].replace(to_replace ={0:"Ingles", 1:'Espanhol'}, inplace=True)
df['Q006'].replace(to_replace = {'A' :'Nenhuma renda.','B' : '1 SM',
                                 'C' :'1 a 1.5 SM','D' : '1.5 a 2 SM',
                                 'E' :'2 a 2.5 SM','F' :'2.5 a 3 SM',
                                 'G' :'3 a 4 SM','H' :'4 a 5 SM',
                                 'I' :'5 a 6 SM','J' :'6 a 7 SM',
                                 'K' :'7 a 8 SM','L' :'8 a 9 SM',
                                 'M' :'9 a 10 SM','N' :'10 a 12 SM',
                                 'O' :'12 a 15 SM','P' :'15 a 20 SM',
                                 'Q' :'Mais de 20 SM'}, inplace=True)

df.rename(columns={'Q006':'RENDA'}, inplace=True)




df.drop(columns=[ 'CO_MUNICIPIO_RESIDENCIA'], inplace=True)
df['RENDA'].replace(to_replace = {'Nenhuma renda.':np.nan, '1 SM': 1,
                                 '1 a 1.5 SM': 1.25, '1.5 a 2 SM': 1.75,
                                 '2 a 2.5 SM': 2.25, '2.5 a 3 SM' : 2.75,
                                 '3 a 4 SM': 3.5, '4 a 5 SM' : 4.5 ,
                                 '5 a 6 SM': 5.5, '6 a 7 SM' : 6.5 ,
                                 '7 a 8 SM': 7.5, '8 a 9 SM': 8.5,
                                 '9 a 10 SM': 9.5, '10 a 12 SM' : 11,
                                 '12 a 15 SM': 13.5, '15 a 20 SM' : 17.5,
                                 'Mais de 20 SM': 20}, inplace=True)

print('Finished loading')

Finished loading


In [47]:
df = df.reset_index()
df.drop(columns=['index'], inplace=True)

In [48]:
df

,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_ESCOLA,TP_LINGUA,RENDA,NOTA_FINAL
0,18.0,F,Branca,Publica,Ingles,1.25,599.52
1,17.0,F,NaN,Publica,Ingles,1.75,614.76
2,18.0,M,Branca,Publica,Espanhol,1.25,433.70
3,18.0,M,Parda,Publica,Ingles,3.50,685.08
4,19.0,F,Parda,Publica,Espanhol,1.25,471.16
...,...,...,...,...,...,...,...
3333,18.0,M,Branca,Publica,Ingles,4.50,535.54
3334,19.0,F,Branca,Publica,Ingles,2.25,426.94
3335,19.0,M,Parda,Publica,Ingles,3.50,436.78
3336,17.0,F,Branca,Publica,Ingles,2.25,505.12


In [ ]:
file_loader = FileSystemLoader("templates")
env = Environment(loader=file_loader)

template = env.get_template("bid.html")

keys, data_array = build_data(df)

output = template.render(data=keys, full_matrix=data_array)

with open("display_tab.html", "w") as result_file:
    result_file.write(output)

In [51]:
file_loader = FileSystemLoader("templates")
env = Environment(loader=file_loader)

template = env.get_template("table.html")

keys, data_array = table_data_format(df)

output = template.render(metadata_table_viz=keys, dataarray_table_viz=data_array)

with open("testing.html", "w") as result_file:
    result_file.write(output)


In [ ]:
fields = [
          'TP_SEXO', 'TP_COR_RACA', 'TP_ESCOLA', 'TP_DEPENDENCIA_ADM_ESC', 'TP_ENSINO', 
          'IN_TREINEIRO', 'TP_LOCALIZACAO_ESC', 'TP_PRESENCA_CN', 
          "TP_PRESENCA_CH", "TP_PRESENCA_LC", 'TP_PRESENCA_MT', "NU_NOTA_CN", 'NU_NOTA_CH', 
          'NU_NOTA_LC', 'NU_NOTA_MT', 'TP_LINGUA', 'TP_STATUS_REDACAO', 'NU_NOTA_REDACAO', 'Q006', 'Q026',
          'IN_BAIXA_VISAO','IN_CEGUEIRA','IN_SURDEZ','IN_DEFICIENCIA_AUDITIVA','IN_SURDO_CEGUEIRA',
          'IN_DEFICIENCIA_FISICA','IN_DEFICIENCIA_MENTAL','IN_DEFICIT_ATENCAO','IN_DISLEXIA',
          'IN_DISCALCULIA','IN_AUTISMO','IN_VISAO_MONOCULAR','IN_OUTRA_DEF',
          'NU_IDADE',
          'CO_MUNICIPIO_RESIDENCIA',
          'Q003','Q004'
         ]
df = pd.read_csv('C:\\Users\\Lucas\\Downloads\\microdados_enem2018\\DADOS\\MICRODADOS_ENEM_2018.csv', 
                 sep=';', encoding='latin-1', skipinitialspace=True, usecols=fields)

df.drop(df[
    (df['IN_TREINEIRO'] == 1) |
    (df['TP_PRESENCA_CN'] != 1) |
    (df['TP_PRESENCA_CH'] != 1) |
    (df['TP_PRESENCA_LC'] != 1) |
    (df['TP_PRESENCA_MT'] != 1) |
    (df['TP_STATUS_REDACAO'] != 1) |
    ((df['Q026'] != 'B') & (df['Q026'] != 'A')) |
    ].index, inplace=True)

df['NOTA_FINAL'] = list(map(lambda a,b,c,d, e: (a + b + c + d + e)/5, df['NU_NOTA_CN'], 
                            df['NU_NOTA_CH'], df['NU_NOTA_LC'], df['NU_NOTA_MT'], df['NU_NOTA_REDACAO']))
df["TP_COR_RACA"].replace(to_replace ={0:np.nan,1:"Branca",2:'Preta',3:"Parda", 4:'Amarela', 5:'Indígena'},
                          inplace=True)
df.drop(columns=['IN_TREINEIRO', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'TP_PRESENCA_CN', 
                 'TP_STATUS_REDACAO', 'Q026', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 
                 'NU_NOTA_REDACAO'], inplace=True)
df["TP_ESCOLA"].replace(to_replace ={1:np.nan, 2:'Publica', 3:"Exterior", 4:'Privada'}, inplace=True)
df["TP_LOCALIZACAO_ESC"].replace(to_replace ={1:"Urbana", 2:'Rural'}, inplace=True)
df["TP_LINGUA"].replace(to_replace ={0:"Ingles", 1:'Espanhol'}, inplace=True)
df['TP_ENSINO'].replace(to_replace = {1: 'Regular', 2: 'Ed Especial', 3:'EJA'}, inplace=True)
df['Q006'].replace(to_replace = {'A' :'Nenhuma renda.','B' : '1 SM',
                                 'C' :'1 a 1.5 SM','D' : '1.5 a 2 SM',
                                 'E' :'2 a 2.5 SM','F' :'2.5 a 3 SM',
                                 'G' :'3 a 4 SM','H' :'4 a 5 SM',
                                 'I' :'5 a 6 SM','J' :'6 a 7 SM',
                                 'K' :'7 a 8 SM','L' :'8 a 9 SM',
                                 'M' :'9 a 10 SM','N' :'10 a 12 SM',
                                 'O' :'12 a 15 SM','P' :'15 a 20 SM',
                                 'Q' :'Mais de 20 SM'}, inplace=True)
df['TP_DEPENDENCIA_ADM_ESC'].replace(to_replace={1:'Federal', 2:'Estadual', 3:'Municipal', 4:'Privada'}, inplace=True)

q1,q2,q3 = df['NOTA_FINAL'].quantile([0.25,0.5,0.75])


df['NOTA_FINAL'] = pd.to_numeric(df['NOTA_FINAL'], errors='ignore')

def quartile_definition(val):
    if type(val) != 'str':
        if val < q1:
            return 'less than 25%'
        if val < q2:
            return 'between 25% and 50%'
        if val < q3:
            return 'between 50% and 75%'
        return 'above 75%'

df['NOTA_FINAL'] = df['NOTA_FINAL'].map(quartile_definition)
df.rename(columns={'Q006':'RENDA'}, inplace=True)

df['ATENCAO_ESPECIAL'] = df['IN_BAIXA_VISAO'] + df['IN_CEGUEIRA'] + df['IN_SURDEZ'] + df['IN_DEFICIENCIA_AUDITIVA'] + df['IN_SURDO_CEGUEIRA'] + df['IN_DEFICIENCIA_FISICA'] + df['IN_DEFICIENCIA_MENTAL'] + df['IN_DEFICIT_ATENCAO'] + df['IN_DISLEXIA'] + df['IN_DISCALCULIA'] + df['IN_AUTISMO'] + df['IN_VISAO_MONOCULAR'] + df['IN_OUTRA_DEF']
df['ATENCAO_ESPECIAL'] = list(map(lambda a: 'sim' if a != 0 else 'não', df['ATENCAO_ESPECIAL']))

df.drop(columns=[
          'IN_BAIXA_VISAO','IN_CEGUEIRA','IN_SURDEZ','IN_DEFICIENCIA_AUDITIVA','IN_SURDO_CEGUEIRA',
          'IN_DEFICIENCIA_FISICA','IN_DEFICIENCIA_MENTAL','IN_DEFICIT_ATENCAO','IN_DISLEXIA',
          'IN_DISCALCULIA','IN_AUTISMO','IN_VISAO_MONOCULAR','IN_OUTRA_DEF'
        ], inplace=True)

df['NU_IDADE'] = pd.cut(df['NU_IDADE'], bins=[0, 10, 15, 18, 20, 22, 25, 30, 40, 50, 100]).astype(str)
df['NU_IDADE'].replace(to_replace={'nan': np.nan}, inplace=True)
def region_of_residency(a):
    if str(a)[0] == '1':
        return 'Norte'
    if str(a)[0] == '2':
        return 'Nordeste'
    if str(a)[0] == '3':
        return 'Sudeste'
    if str(a)[0] == '4':
        return 'Sul'
    if str(a)[0] == '5':
        return 'Centro-Oeste'
df['CO_MUNICIPIO_RESIDENCIA'] = list(map(region_of_residency,df['CO_MUNICIPIO_RESIDENCIA']))

df.rename(columns={'CO_MUNICIPIO_RESIDENCIA':'REGIAO_RESIDENCIA'}, inplace=True)

df['Q003'].replace(to_replace={'A' : 'Grupo 1','B' : 'Grupo 2','C' : 'Grupo 3','D' : 'Grupo 4','E' : 'Grupo 5','F' : np.nan}, inplace=True)
df['Q004'].replace(to_replace={'A' : 'Grupo 1','B' : 'Grupo 2','C' : 'Grupo 3','D' : 'Grupo 4','E' : 'Grupo 5','F' : np.nan}, inplace=True)

df.rename(columns={'Q003':'TRABALHO_PAI', 'Q004':'TRABALHO_MAE'}, inplace=True)


print('Finished loading')

In [ ]:
import pandas as pd
import numpy as np
import pandas_profiling
columns = ["symboling","normalized-losses","make","fuel-type","aspiration","num-of-doors",
           "body-style","drive-wheels","engine-location","wheel-base","length","width",
           "height","curb-weight","engine-type","num-of-cylinders","engine-size",
           "fuel-system","bore","stroke","compression-ratio","horsepower","peak-rpm",
           "city-mpg","highway-mpg","price"]
df = pd.read_csv('imports-85.data', names=columns)
pandas_profiling.ProfileReport(df).to_file("Desktop/example_pandas_profiling_modified.html")